# Combined CCLE Model Training

This notebook combines the CCLE gene expression dataset with dynamic features and then is trained together

## Initialisation

## Negative Control (make_regression data)

To ensure all models are working as intended

## Positive Control (CCLE Only)

## Combined (CCLE + Dyn)